In [1]:
import sys
sys.path.append("..")
from MPRA_predict.utils import *
from MPRA_predict.datasets import *
from MPRA_predict.metrics import *

In [2]:
df = pd.read_csv('../data/Sei/Sei_tracks_info.csv')

df_pivot = df.pivot_table(
    values="index", 
    index="cell_type", 
    columns="assay", 
    aggfunc=list,
)
df_pivot = df_pivot.map(lambda x: x if isinstance(x, list) else [])
df_pivot

assay                    ADNP AEBP2 AFF1 AFF4 AGO1 AGO2 AHR AHRR AP-2alpha AP-2gamma APOBEC3B  AR ARID1A ARID2 ARID3A ARID5B ARNT ARNT2 ARNTL ARRB1 ASCL1 ASCL2 ASH1L ASH2L ASXL1 ATAC-seq ATF1 ATF2 ATF3 ATF4 ATF5 ATF7IP ATOH1 ATRX AUTS2 BACH1 BACH2 BAF155 BAF170 BAHD1 BANF1 BANP BARX1 BARX2 BATF BATF3 BCAT1 BCL11A BCL11B BCL3 BCL6 BCLAF1 BCOR BDP1 BHLHE40 BIRA BMI1 BPTF BRAF BRCA1 BRD1 BRD2    BRD3    BRD4 BRD7 BRD9 BRDU BRF1 BRF2 BRPF3 BTAF1 Bach1 Brg1 C11orf30 C17orf49 C17orf96 CAMTA2 CASP8AP2 CBFA2T2 CBFB CBX1 CBX2 CBX3 CBX4 CBX5 CBX6 CBX7 CBX8 CCDC101 CCND2 CCNT2 CD59 CD74 CDC5L CDCA2 CDK12 CDK2 CDK6 CDK7 CDK8 CDK9 CDX2 CEBPA CEBPB CEBPD CEBPG CEBPZ CENPA CENPC CENPT CHAMP1 CHAT CHD1 CHD2 CHD4 CHD5 CHD7 CHD8 CHRM2 CIITA CLOCK CNOT3 COPS2 COREST CPSF3L CREB1 CREB3 CREB3L4 CREBBP CREM CRY1 CSDE1 CSNK2A1 CTBP1 CTBP2 CTCF CTCFL CTNNB1 CUX1 CtBP2 DACOR1 DAXX DCP1A DDX20 DDX21 DDX5 DEAF1 DEK DIDO1 DLX1 DLX2 DMC1 DNMT1 DNMT3A DNMT3B    DNase DNase.all.peaks DNase.fdr0.01.hot  \
cell_type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
1015c                      []    []   []   []   []   []  []   []        []        []       []  []     []    []     []     []   []    []    []    []    []    []    []    []    []       []   []   []   []   []   []     []    []   []    []    []    []     []     []    []    []   []    []    []   []    []    []     []     []   []   []     []   []   []      []   []   []   []   []    []   []   []      []  [8996]   []   []   []   []   []    []    []    []   []       []       []       []     []       []      []   []   []   []   []   []   []   []   []   []      []    []    []   []   []    []    []    []   []   []   []   []   []   []    []    []    []    []    []    []    []    []     []   []   []   []   []   []   []   []    []    []    []    []    []     []     []    []    []      []     []   []   []    []      []    []    []   []    []     []   []    []     []   []    []    []    []   []    []  []    []   []   []   []    []     []     []       []              []                []   
10326                      []    []   []   []   []   []  []   []        []        []       []  []     []    []     []     []   []    []    []    []    []    []    []    []    []       []   []   []   []   []   []     []    []   []    []    []    []     []     []    []    []   []    []    []   []    []    []     []     []   []   []     []   []   []      []   []   []   []   []    []   []   []  [8997]      []   []   []   []   []   []    []    []    []   []       []       []       []     []       []      []   []   []   []   []   []   []   []   []   []      []    []    []   []   []    []    []    []   []   []   []   []   []   []    []    []    []    []    []    []    []    []     []   []   []   []   []   []   []   []    []    []    []    []    []     []     []    []    []      []     []   []   []    []      []    []    []   []    []     []   []    []     []   []    []    []    []   []    []  []    []   []   []   []    []     []     []       []              []                []   
1184_Stem_cell             []    []   []   [

In [3]:
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'H3K27me3', 'H3K4me1', 'H3K36me3', 'H3K9me3', 'CTCF', 'ATAC-seq']

cell_types = ['LCL', 'HepG2']
cell_standard_names = ['LCL_Lymphoblastoid_Blood', 'HepG2_Hepatocellular_Carcinoma']

In [4]:
df_track = pd.DataFrame(index=cell_types, columns=assays)

for i, cell_type in enumerate(cell_types):
    cell_standard_name = cell_standard_names[i]
    df_track.loc[cell_type] = df_pivot.loc[cell_standard_name]

df_track.map(len)

,DNase,H3K4me3,H3K27ac,H3K27me3,H3K4me1,H3K36me3,H3K9me3,CTCF,ATAC-seq
LCL,0,4,9,5,1,0,0,0,19
HepG2,9,8,9,2,4,3,2,17,0


In [6]:
pred_array = load_h5('outputs/Tewhey_MPRA_LCL_Sei_pred.h5')
print(pred_array.shape)

(77281, 21907)


In [9]:
MPRA_df = pd.read_csv('../data/Tewhey_MPRA_LCL/Tewhey_MPRA.csv')
print(MPRA_df.shape)

(77281, 4)


In [10]:
# average every channel

pearson_df = pd.DataFrame(index=cell_types, columns=assays, dtype=float)

for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        indice = df_track.loc[cell_type, assay]

        if len(indice) > 0:
            pred = logit(pred_array[:, indice]).mean(1)
            true = MPRA_df[cell_type]
            r, _ = pearson(pred, true)
            pearson_df.loc[cell_type, assay] = r
            MPRA_df[f'{cell_type}_{assay}'] = pred
        else:
            MPRA_df[f'{cell_type}_{assay}'] = np.nan

pearson_df

,DNase,H3K4me3,H3K27ac,H3K27me3,H3K4me1,H3K36me3,H3K9me3,CTCF,ATAC-seq
LCL,NaN,0.210,0.284,0.126,0.291,NaN,NaN,NaN,0.291
HepG2,0.075,0.095,0.090,-0.048,0.019,0.003,-0.213,0.043,NaN


In [ ]:
MPRA_df.to_csv('../data/Tewhey_MPRA_LCL/Tewhey_MPRA_LCL_Sei_feature_0512.csv', index=False)